<a href="https://colab.research.google.com/github/AndersOMadsen/PhAI/blob/main/PhAI_phase_determination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PhAI phase determination notebook.

In the cell below, please supply the name of your .hkl data file.
Also, indicate the number of phasing cycles you would like to use. The default number is 5 and should be sufficient in almost all cases.
You can also choose to have all phases equal zero as your starting guess, or use random phases.
PhAI will output a '.F' file containing the structure factors, including phases.
Provided the cell dimensions of your system, we will generate and XPLOR map (electron density map) that can be read by the program PyMol.
We also generate a '.fcf' cif file containing the phased structure factors. The format corresponds to the 'LIST 3' format produced by shelxl, and is readable by Olex2.


The current version of this notebook is hardcoded to the space group P21/c. This implies that if your data corresponds to an other space group, or to another setting of spacegroup no. 14, such as P21/a, you will have to transform the data.


### -- User input --

In [11]:
infile = 'COD_2016452.hkl'  # input hkl file.
n = 5                 # number of phasing cycles
p = 1                 # p = 0 => initial phases zero. p = 1 => initial phases random.
t = False             # should we output the results of each phase cycle?
cellparam = [9.748,8.89,7.566,90,112.74,90]  # cell parameters
map_resolution = 0.1  # resolution of XPLOR map

### -- End of user input --

In [12]:
!pip install einops
!pip install xraydb
!pip install fortranformat
!pip install --upgrade --no-cache-dir gdown
!pip install torch

# downloading sample data, PhAI code and PhAI network.
!gdown 1_eleZ6dBvdKQQeZwxeOJ82g5lPVzmb2M
!gdown 14lqkA_Frfy8WpoYyJ-v2sfKkhfPTlNFO
!gdown 10U-JUhNQKvoYCRPAv5k-iC2D5vdq6MxM
!gdown 1Str3GWahzB1QZtpU2obBj-KSbH9JCV8P




  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [13]:

import PhAI_jupyter_test as PJ
import pandas as pd
import crystallography_module
import torch
import math
import numpy as np



Symmetry operators (P21/c)

In [14]:
symmR = [[[ 1. , 0.,  0.],
          [ 0. , 1.,  0.],
          [ 0. , 0.,  1.]],

         [[-1.,  0.,  0.],
          [ 0.,  1.,  0.],
          [ 0.,  0., -1.]],

         [[ 1.,  0.,  0.],
          [ 0., -1.,  0.],
          [ 0.,  0.,  1.]],

         [[-1.,  0.,  0.],
          [ 0., -1.,  0.],
          [ 0.,  0., -1.]]]

symmT = [[0.,  0.,  0. ],
         [0.,  0.5, 0.5],
         [0.,  0.5, 0.5],
         [0.,  0.,  0. ]]

Defining and loading the neural network and data array.

In [15]:

# model definition
model_args = {
     'max_index' : 10,
       'filters' : 96,
   'kernel_size' : 3,
     'cnn_depth' : 6,
           'dim' : 1024,
       'dim_exp' : 2048,
 'dim_token_exp' : 512,
     'mlp_depth' : 8,
   'reflections' : 1205,
}


model = PJ.PhAINeuralNetwork(**model_args)
state = torch.load('./PhAI_model.pth')#, weights_only = True)
model.load_state_dict(state)

max_index = 10
hkl_array = []
for h in range(-max_index, max_index+1):
    for k in range(0, max_index+1):
        for l in range(0, max_index+1):
            if not(h==0 and k==0 and l==0):
                if math.sqrt(h**2+k**2+l**2) <= max_index:
                    hkl_array.append([h,k,l])
hkl_array = np.array(hkl_array,dtype=np.int32)



Loading and sorting of reflections

In [16]:
data = pd.read_table(infile, header=None, delim_whitespace=True)
H_tmp = data.loc[:,0:2].astype(int).to_numpy()
Fabs_tmp = data.loc[:,3].astype(float).to_numpy()

H, Fabs = crystallography_module.merge_reflections(H_tmp, Fabs_tmp)


amplitudes = torch.zeros(1,21,11,11)
for i in range(len(H)):
  if H[i][0] + 10 < 21:
    if H[i][1] < 11:
      if H[i][2] < 11:
        amplitudes[0][H[i][0]+10][H[i][1]][H[i][2]] = Fabs[i]


amplitudes_ord = []
for h in range(-max_index, max_index+1):
    for k in range(0, max_index+1):
        for l in range(0, max_index+1):
            if not(h==0 and k==0 and l==0):
                if math.sqrt(h**2+k**2+l**2) <= max_index:
                    amplitudes_ord.append(amplitudes[0][h+10][k][l])



Application of PhAI and output of results (.F and .fcf format)

In [17]:


if p == 0:
    init_phases = torch.zeros(1,21,11,11)
else:
    init_phases = PJ.randomize_output(torch.zeros(1,21,11,11))

for i in range(n):
    #print('cycle: ', i+1)
    if i == 0:
        output = PJ.phases(PJ.model(amplitudes, init_phases))
        if t == True and n != 1:
            PJ.output_files(amplitudes_ord, output, infile[:len(infile)-4] + '_' + str(i+1) + '.F', infile[:len(infile)-4] + '_phase_extension_' + str(i+1) + '.F', cellparam)
    else:
        for j in range(len(PJ.hkl_array)):
            init_phases[0][PJ.hkl_array[j][0]+10][PJ.hkl_array[j][1]][PJ.hkl_array[j][2]] = output[0][j]
        output = PJ.phases(PJ.model(amplitudes, init_phases))
        if t == True and i + 1 != n:
            PJ.output_files(amplitudes_ord, output, infile[:len(infile)-4] + '_' + str(i+1) + '.F', infile[:len(infile)-4] + '_phase_extension_' + str(i+1) + '.F', cellparam)

PJ.output_files(amplitudes_ord, output, infile[:len(infile)-4] + '.F', infile[:len(infile)-4] + '_phase_extension.F',cellparam)


Reading data from saved .F file, computing density and saving as .xplor file. Can be loaded by Pymol.
The .fcf file can be loaded into Olex2 for map visualization.

In [18]:
H, F = crystallography_module.read_F(infile[:len(infile)-4] + '.F')

In [19]:
H_full, F_full = crystallography_module.complete_hkl(H, F, SG_symm=[symmR, symmT], half=True)

In [20]:
den_map = crystallography_module.calc_density_map_full(H_full, F_full, cellparam, map_resolution, pixel_mult=(2, 2, 2), N=2, sort_reflns=False)

In [21]:
crystallography_module.save_den_map_xplor(den_map, cellparam, infile[:len(infile)-2] + '.xplor', infile[:len(infile)-2])

End of PhAI Jupyter notebook.